This notebook merges the NCES data with the BoardDocs URLs scrapped from deliverable 1.

Input
- `../deliverable1/release/deliverable_1.csv`
- `../nces2324.csv`

Output
-  `release/deliverable_2.csv`

In [105]:
import pandas as pd

boarddocs_df = pd.read_csv("../deliverable1/release/deliverable_1.csv")
nces_df = pd.read_csv("../nces2324.csv")

/var/folders/cp/kz443md17ndf52c8_802gy080000gn/T/ipykernel_84246/163765961.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nces_df = pd.read_csv("../nces2324.csv")


In [106]:
# select relevant columns
nces_df = nces_df[["LEA_NAME", "LEAID", "LSTREET1", "LSTREET2","LSTREET3","LCITY","LSTATE", "LZIP","PHONE","WEBSITE"]]

In [107]:
# since the most coverage is by the website, let's do a merge there first
# our goal is to match as much boarddocs as possible
# so boarddocs_df is on the left
nces_df = nces_df.rename(columns={"WEBSITE":"home_website"})

In [108]:
nces_df["home_website"].isna().value_counts()

home_website
False    16957
True      2680
Name: count, dtype: int64

In [109]:
# before merge, let's clean
# remove https and www
url_prefix_pattern = r"https?://(www.)?"
nces_df["home_website"] = nces_df["home_website"].str.replace(url_prefix_pattern, '', regex=True)
boarddocs_df["home_website"] = boarddocs_df["home_website"].str.replace(url_prefix_pattern, '', regex=True)

# also remove the trailing /
single_slash_pattern = r"[?<!/]/[?!/]"
nces_df["home_website"] = nces_df["home_website"].str.split(single_slash_pattern,n=1).str[0]
boarddocs_df["home_website"] = boarddocs_df["home_website"].str.split(single_slash_pattern,n=1).str[0]

In [127]:
# check if the websites are duplicated
sum(nces_df[~nces_df["home_website"].isna()]["home_website"].duplicated(keep=False)) / nces_df.shape[0] * 100

6.752558944849009

In [129]:
sum(nces_df[~nces_df["home_website"].isna()]["home_website"].duplicated(keep=False))

1326

In [128]:
# check if the websites are duplicated
sum(boarddocs_df[~boarddocs_df["home_website"].isna()]["home_website"].duplicated(keep=False)) / boarddocs_df.shape[0] * 100

1.3174404015056462

In [130]:
sum(boarddocs_df[~boarddocs_df["home_website"].isna()]["home_website"].duplicated(keep=False))

21

In [134]:
boarddocs_df.loc[(~boarddocs_df["home_website"].isna()) & (boarddocs_df["home_website"].duplicated(keep=False)),:].sort_values(by="home_website")

,URL,school_district,address,home_website,phone
322,https://go.boarddocs.com/nj/bergen/Board.nsf/P...,Bergen County Technical Schools,"540 Farview Ave. , Paramus , NJ 07652",bergen.org,NaN
1781,https://go.boarddocs.com/nj/bergencss/Board.ns...,Bergen County Special Services,"540 Farview Avenue, Paramus, NJ 07652",bergen.org,NaN
2275,https://go.boarddocs.com/oh/cmsd/Board.nsf/Public,Cleveland Metropolitan School District,NaN,clevelandmetrtoschools.org,NaN
2891,https://go.boarddocs.com/OH/CMSD/Board.nsf/Public,Cleveland Metropolitan School District,NaN,clevelandmetrtoschools.org,NaN
3133,https://go.boarddocs.com/mi/scacad/Board.nsf/P...,Saginaw Covenant Academy,"508 S. Washington Avenue | Saginaw, MI 48607 |...",covenantacademies.org,(989) 596-1100
3777,https://go.boarddocs.com/mi/kcacad/Board.nsf/P...,Kalamazoo Covenant Academy,"400 W Crosstown Pkwy | Kalamazoo, MI 49001 | 2...",covenantacademies.org,(269) 888-2700
483,https://go.boarddocs.com/mi/mcacad/Board.nsf/P...,Muskegon Covenant Academy,"125 Catherine Avenue | Muskegon, MI 49442 | 23...",covenantacademies.org,(231) 720-3100
1876,https://go.boarddocs.com/mi/macki/Board.nsf/Pu...,NaN,NaN,eupschools.org,NaN
2370,https://go.boarddocs.com/mi/engadine/Board.nsf...,Engadine Consolidated Schools,NaN,eupschools.org,NaN
657,https://go.boarddocs.com/mi/lca/Board.nsf/Public,Livingston Classical Academy,"8877 Main St., Whitmore Lake, MI 48189 Phone: ...",livingstonclassicalacademy.org,(734) 449-2052


In [110]:
df = pd.merge(boarddocs_df, nces_df[~nces_df["home_website"].isna()], how="left", on="home_website")

In [111]:
# let's check how good is the match
# check how many rows have LEAID
df[~df["LEAID"].isna()].shape[0] / df.shape[0] * 100

60.0200652119388

In [112]:
# ok we have matched 37% of them
# check those that weren't matched but has website

df[(~df["home_website"].isna()) & (df["LEAID"].isna())].sample(5)

,URL,school_district,address,home_website,phone,LEA_NAME,LEAID,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,PHONE
1569,https://go.boarddocs.com/md/msde/Board.nsf/Public,Maryland State Board of Education,"200 West Baltimore Street, Baltimore, MD 21201...",marylandpublicschools.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,https://go.boarddocs.com/oh/strasoh/Board.nsf/...,NaN,Strasburg-Franklin Local Schools | 140 North B...,strasburg.k12.oh.us,(330) 878-5571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621,https://go.boarddocs.com/la/ipssla/Board.nsf/P...,Iberia Parish School System,"1500 Jane Street • P. O. Box 200 • New Iberia,...",iberiaschools.org,(337) 365-2341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2578,https://go.boarddocs.com/oh/fhocking/Board.nsf...,Federal Hocking Local School District,"8461 State Route 144, Stewart, OH 45778 | (740...",fedhock.com,(740) 662-6691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,https://go.boarddocs.com/sc/spart7/Board.nsf/P...,Spartanburg School District 7,"610 Dupre Drive | Spartanburg, SC 29307 | Tel:...",spartanburg7.org,(864) 594-4400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
# let's take out the ones matched first
matched = df[~df["LEAID"].isna()]
boarddocs_df = boarddocs_df[~boarddocs_df["URL"].isin(matched["URL"])]

In [114]:
# match by phone
# then hopefully that can tell us what is up with the website
nces_df = nces_df.rename(columns={
    "PHONE": "phone"
})

In [115]:
# check if all is in same format
phone_pattern = r"^\(\d{3}\)\d{3}\-\d{4}$"
nces_df["phone_good_pattern"] = nces_df["phone"].str.contains(phone_pattern, regex=True, na=False)

In [116]:
nces_df["phone_good_pattern"].value_counts()

phone_good_pattern
True    19637
Name: count, dtype: int64

In [117]:
# let's just add space
nces_df["phone"] = nces_df["phone"].str.replace(")",") ")

In [118]:
# make sure both are unique
sum(nces_df["phone"].isna())

0

In [119]:
# make sure both are unique
sum(nces_df["phone"].duplicated(keep=False))

1453

In [120]:
nces_df.shape

(19637, 11)

In [121]:
# seems like almost 10% of the phone numbers in nces is duplicated
nces_df[nces_df["phone"].duplicated(keep=False)].sort_values(by="phone").head()

,LEA_NAME,LEAID,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,phone,home_website,phone_good_pattern
11110,Bergen County Vocational Technical School Dist...,3401470,540 Farview Avenue,NaN,NaN,Paramus,NJ,7652,(201) 343-6000,bcts.bergen.org,True
11109,Bergen County Special Services School District,3401450,540 Farview Avenue,NaN,NaN,Paramus,NJ,7652,(201) 343-6000,bcss.bergen.org,True
11070,Hudson Arts and Science Charter School,3400787,131 Midland Ave,NaN,NaN,Kearny,NJ,7032,(201) 773-9140,hudsoncharter.org,True
11023,Bergen Arts and Science Charter School,3400715,200 MacArthur Ave,NaN,NaN,Garfield,NJ,7026,(201) 773-9140,bergencharter.org,True
3802,Regional School District 09,903780,654 Morehouse Road,NaN,NaN,Easton,CT,6612,(203) 261-2513,er9.org,True


In [122]:
# merge it
df = pd.merge(boarddocs_df, nces_df, how="left", on="phone")

In [123]:
# check matches
# check how many rows have LEAID
df[~df["LEAID"].isna()].shape[0] / df.shape[0] * 100

41.95544554455445

In [124]:
df[~df["LEAID"].isna()].sample(5)

,URL,school_district,address,home_website_x,phone,LEA_NAME,LEAID,LSTREET1,LSTREET2,LSTREET3,LCITY,LSTATE,LZIP,home_website_y,phone_good_pattern
531,https://go.boarddocs.com/ny/gcsny/Board.nsf/Pu...,Goshen Central School District,"227 Main Street Goshen, NY 10924 | (845) 615-6720",gcsny.org,(845) 615-6720,GOSHEN CENTRAL SCHOOL DISTRICT,3612330.0,227 MAIN ST,NaN,NaN,GOSHEN,NY,10924.0,goshenschoolsny.org,True
755,https://go.boarddocs.com/oh/tcsoh/Board.nsf/Pu...,Tallmadge City School District,"486 East Avenue | Tallmadge, OH 44278 | Ph...",tallmadgeschools.org,(330) 633-3291,Tallmadge City,3904488.0,486 East Ave,NaN,NaN,Tallmadge,OH,44278.0,tallmadgeschools.org/,True
528,https://go.boarddocs.com/vsba/lunenburg/Board....,Lunenburg County Public Schools,"1009 Main St | Kenbridge, VA 23944 | 434.676.2...",lunenburgcountyschools.org,(434) 676-2467,Lunenburg County Public Schools,5102310.0,1009 Main Street,NaN,NaN,Kenbridge,VA,23944.0,lunenburgcountyschools.org/,True
113,https://go.boarddocs.com/wa/pasd/Board.nsf/Public,Port Angeles School District,"905 West 9th Street • Port Angeles, Washington...",portangelesschools.org,(360) 457-8575,Port Angeles School District,5306820.0,905 W 9TH ST,NaN,NaN,PORT ANGELES,WA,98363.0,NaN,True
171,https://go.boarddocs.com/pa/parkland/Board.nsf...,Parkland School District,1210 Springhouse Road • Allentown PA 18104 • P...,parklandsd.org,(610) 351-5503,Parkland SD,4218510.0,1210 Springhouse Rd,NaN,NaN,Allentown,PA,18104.0,parklandsd.org/,True


In [125]:
# this matched 42% of the remaining ones
# add them to the match doc